# R/Chapter 2 - Multiple regression

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simulation-with-multiple-explanatory-variables" data-toc-modified-id="Simulation-with-multiple-explanatory-variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simulation with multiple explanatory variables</a></span></li><li><span><a href="#Matrix-algebra-of-short-regression" data-toc-modified-id="Matrix-algebra-of-short-regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Matrix algebra of short regression</a></span></li><li><span><a href="#Understandng-multicollinearity-with-R" data-toc-modified-id="Understandng-multicollinearity-with-R-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Understandng multicollinearity with R</a></span></li><li><span><a href="#NLSM-data" data-toc-modified-id="NLSM-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>NLSM data</a></span></li><li><span><a href="#OLS-estimates-of-return-to-schooling" data-toc-modified-id="OLS-estimates-of-return-to-schooling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>OLS estimates of return to schooling</a></span></li><li><span><a href="#Simulation-of-dual-path-model" data-toc-modified-id="Simulation-of-dual-path-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Simulation of dual path model</a></span></li><li><span><a href="#Dual-path-estimator-vs-long-regression" data-toc-modified-id="Dual-path-estimator-vs-long-regression-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Dual path estimator vs long regression</a></span></li></ul></div>

In [1]:
# R in english
Sys.setenv(LANG = "en")

## Simulation with multiple explanatory variables

In [2]:
# generates simulated data
set.seed(123456789)
N <- 1000
a <- 2
b <- 3
c <- 4
u_x <- rnorm(N)
alpha <- 0
x <- x1 <- (1-alpha)*runif(N)+alpha*u_x
w <- w1 <- (1-alpha)*runif(N)+alpha*u_x
# creates two identitical variables
# x1 and w1 are used later
u <- rnorm(N)
y <- a + b*x + c*w + u

# regressions
lm1 <- lm(y~x)
lm2 <- lm(y~x+w)

In [3]:
# we repeat for a different version of alpha
# allows for dependance between x and w
alpha <- 0.5
x <- x2 <- (1-alpha)*runif(N)+alpha*u_x
w <- w2 <- (1-alpha)*runif(N)+alpha*u_x
y <- a + b*x + c*w + u

# regressions
lm3 <- lm(y~x)
lm4 <- lm(y~x+w)

In [4]:
# and with x and w highly correlated 
# i.e. multicollinearity 
alpha <- 0.95
x <- x3 <- (1-alpha)*runif(N)+alpha*u_x
w <- w3 <- (1-alpha)*runif(N)+alpha*u_x
y <- a + b*x + c*w + u

# regressions
lm5 <- lm(y~x)
lm6 <- lm(y~x+w)

In [6]:
#install.packages("stargazer")
require(stargazer)

# presenting regression results
stargazer(list(lm1, lm2, lm3, lm4, lm5, lm6),
         keep.stat = c("n", "rsq"),
         float = FALSE, font.size = "small", digits = 2,
         keep = c(1:6), type="text")
# stargazer renders properly in Jupyter with the option
# type="text"


                           Dependent variable:              
             -----------------------------------------------
                                    y                       
               (1)     (2)     (3)     (4)     (5)     (6)  
------------------------------------------------------------
x            3.14*** 2.81*** 6.84*** 2.86*** 7.01***  0.67  
             (0.17)  (0.11)  (0.08)  (0.17)  (0.03)  (1.59) 
                                                            
w                    4.05***         4.16***         6.35***
                     (0.11)          (0.16)          (1.59) 
                                                            
Constant     3.98*** 2.15*** 2.14*** 2.07*** 2.07*** 2.08***
             (0.10)  (0.08)  (0.05)  (0.04)  (0.03)  (0.03) 
                                                            
------------------------------------------------------------
Observations  1,000   1,000   1,000   1,000   1,000   1,000 
R2            0.26    0

## Matrix algebra of short regression

In [9]:
cov(x1, w1)
cov(x2, w2)
t(x2)%*%w2
# measures the correlation between the Xs and the Ws.

[1] 0.007019082

[1] 0.2557656

318.8261


## Understandng multicollinearity with R

In [11]:
X2 <- cbind(1, x3, w3)
solve(t(X2)%*%X2)%*%t(X2)%*%u

,0.0766164
x3,-2.3321265
w3,2.3462160


In [15]:
mean(u)
cov(x3,u)
cov(w3,u)
(1/N)*t(X2)%*%u

[1] 0.07635957

[1] 0.01316642

[1] 0.01413717

,0.07635957
x3,0.01593319
w3,0.01687792


In [16]:
1/det(t(X2)%*%X2)
# calculates the reciprocal of the determnant of the matrixx

[1] 2.610265e-06

## NLSM data

In [18]:
x <- read.csv("../data/nls.csv", as.is = TRUE)

In [21]:
# let's have a look a the data
head(x)

,id,nearc2,nearc4,nearc4a,nearc4b,ed76,ed66,age76,daded,nodaded,⋯,noint80,enroll76,enroll78,enroll80,kww,iq,marsta76,marsta78,marsta80,libcrd14
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,⋯,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2,0,0,0,0,7,5,29,9.94,1,⋯,0,0,0,0,15,.,1,1,1,0
2,3,0,0,0,0,12,11,27,8.00,0,⋯,0,0,0,0,35,93,1,4,4,1
3,4,0,0,0,0,12,12,34,14.00,0,⋯,1,0,.,.,42,103,1,.,.,1
4,5,1,1,1,0,11,11,27,11.00,0,⋯,0,0,.,0,25,88,1,.,5,1
5,6,1,1,1,0,12,12,34,8.00,0,⋯,1,0,0,.,34,108,1,1,.,0
6,7,1,1,1,0,12,11,26,9.00,0,⋯,0,0,0,0,38,85,1,4,4,1


In [23]:
x$wage76 <- as.numeric(x$wage76)
x$lwage76 <- as.numeric(x$lwage76)

x1 <- x[is.na(x$lwage76)==0,]
x1$exp <- x1$age76 - x1$ed76 - 6
# working years after school
x1$exp2 <- (x1$exp^2)/100
# experienced squared divided by 100

## OLS estimates of return to schooling

In [24]:
lm1 <- lm(lwage76~ed76, data = x1)
lm2 <- lm(lwage76~ed76 + exp + exp2, data = x1)
lm3 <- lm(lwage76~ed76 + exp + exp2 + black + reg76r, data = x1)
lm4 <- lm(lwage76~ed76 + exp + exp2 + black + reg76r
          + smsa76r + smsa66r + reg662 + reg663 + reg664
          + reg665 + reg666 + reg667 + reg668 + reg669,
          data = x1)
# reg76 refers to living in the South in 1976
# smsa refers whether they are urban or rural in 1976
# reg refers to regions of the US (North, South, East, West)
# 66 refers to 1966

In [36]:
# presenting regression results
stargazer(list(lm1, lm2, lm3, lm4),
         keep.stat = c("n", "rsq"),
         float = FALSE, font.size = "small", digits = 3,
         keep = c(1:6), type="text")


                      Dependent variable:          
             --------------------------------------
                            lwage76                
               (1)       (2)       (3)       (4)   
---------------------------------------------------
ed76         0.052*** 0.093***  0.078***  0.075*** 
             (0.003)   (0.004)   (0.004)   (0.003) 
                                                   
exp                   0.090***  0.085***  0.085*** 
                       (0.007)   (0.007)   (0.007) 
                                                   
exp2                  -0.249*** -0.234*** -0.229***
                       (0.034)   (0.032)   (0.032) 
                                                   
black                           -0.178*** -0.199***
                                 (0.018)   (0.018) 
                                                   
reg76r                          -0.150*** -0.148***
                                 (0.015)   (0.026) 
           

## Simulation of dual path model

In [39]:
set.seed(123456789)
N <- 50
a <- 1
b <- 0
c <- 3
d <- 4
x <- round(runif(N)) # creates a vector of 0s and 1s
u_w <- runif(N)
w <- d*x + u_w
u <- rnorm(N)
y <- a + b*x + c*w + u

In [51]:
require(xtable)
# results for a short regression
lm1 <- lm(y~x)
xtable(summary(lm1))

,Estimate,Std. Error,t value,Pr(>|t|)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.586659,0.2977401,8.687639,2.053368e-11
x,12.240605,0.4128912,29.646076,1.626472e-32


In [79]:
library(IRdisplay)
lm2 <- lm(y~x + w)

# results for a long regression
display(xtable(summary(lm2)))

,Estimate,Std. Error,t value,Pr(>|t|)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5644028,0.2811701,2.007336,5.048349e-02
x,-5.1480643,1.8846525,-2.731572,8.849763e-03
w,4.2854065,0.4604478,9.307042,3.085038e-12


In [74]:
# in case we want the LATEX output
# for a nice table
print(xtable(lm2, type = "latex"))

% latex table generated in R 4.0.0 by xtable 1.8-4 package
% Sun Feb  7 22:13:03 2021
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & Estimate & Std. Error & t value & Pr($>$$|$t$|$) \\ 
  \hline
(Intercept) & 0.5644 & 0.2812 & 2.01 & 0.0505 \\ 
  x & -5.1481 & 1.8847 & -2.73 & 0.0088 \\ 
  w & 4.2854 & 0.4604 & 9.31 & 0.0000 \\ 
   \hline
\end{tabular}
\end{table}


In [83]:
e_hat <- lm(y~x)$coef[2]
# Coeff 2 is the slope of the element of interest
c_hat <- lm(y~w)$coef[2]
d_hat <- lm(w~x)$coef[2]

# estimate of b
e_hat - c_hat*d_hat

x 
-0.08876039

## Dual path estimator vs long regression

In [87]:
set.seed(123456789)
b_mat <- matrix(NA, 100, 3)
for (i in 1:100){
    x <- round(runif(N))
    u_w <- runif(N)
    w <- d*x + u_w
    u <- rnorm(N)
    y <- a + b*x + c*w + u
    lm2_temp <- summary(lm(y~x+w))
    b_mat[i,1] <- lm2_temp[[4]][2]
    # the fourth element in the list is the result vector
    # the second item of that vector is the coefficient on x
    b_mat[i,2] <- lm2_temp[[4]][8]
    # the 8th item is the T-stat on the coefficient on x
    e_hat <- lm(y~x)$coef[2]
    c_hat <- lm(y~w)$coef[2]
    d_hat <- lm(w~x)$coef[2]
    b_mat[i,3] <- e_hat - c_hat*d_hat
}

In [89]:
colnames(b_mat) <- c("Standard Est.", "T-Stat of Standard", "Proposed Est.")
summ_tab <- summary(b_mat)
row_names <- NULL
display(xtable(summ_tab, floating = FALSE))

,Standard Est.,T-Stat of Standard,Proposed Est.
,<chr>,<chr>,<chr>
X,Min. :-5.1481,Min. :-3.02921,Min. :-0.119608
X.1,1st Qu.:-1.6962,1st Qu.:-0.80145,1st Qu.:-0.031123
X.2,Median :-0.3789,Median :-0.19538,Median :-0.008578
X.3,Mean :-0.1291,Mean :-0.07418,Mean :-0.002654
X.4,3rd Qu.: 1.3894,3rd Qu.: 0.74051,3rd Qu.: 0.030679
X.5,Max. : 6.5594,Max. : 2.68061,Max. : 0.104119
